In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
merchants = pd.read_excel('./merchants.xlsx')
transactions = pd.read_excel('./transactions.xlsx')
transactions = transactions[['merchantid', 'cost', 'countertype',
                             'date', 'fraudbits', 'model_cost']]

In [3]:
trading = merchants.merge(transactions[transactions.fraudbits == 0], how='right')
trading.head(3)

,coef,guild,merchantid,cost,countertype,date,fraudbits,model_cost
0,0.46,Гильдия Парфюмеров,100299578,7.39,buy,924-09-27 12:57,0,30.0
1,0.57,Ассоциация Оливковых Торговцев,445135,22.47,sell,924-09-27 15:48,0,80.0
2,0.86,Гильдия Обработчиков Шерсти,57512,9.81,buy,924-09-27 18:51,0,690.0


Calculate the amnesty coefficient for each operation
$$\frac{|cost-model\_cost|}{coef}$$

In [4]:
trading['amnesty'] = abs(trading.cost - trading.model_cost) / trading.coef
trading.head()

,coef,guild,merchantid,cost,countertype,date,fraudbits,model_cost,amnesty
0,0.46,Гильдия Парфюмеров,100299578,7.39,buy,924-09-27 12:57,0,30.0,49.152174
1,0.57,Ассоциация Оливковых Торговцев,445135,22.47,sell,924-09-27 15:48,0,80.0,100.929825
2,0.86,Гильдия Обработчиков Шерсти,57512,9.81,buy,924-09-27 18:51,0,690.0,790.918605
3,0.42,Гильдия Обработчиков Шерсти,90356927,6.21,buy,924-09-27 20:05,0,32.5,62.595238
4,0.50,Ассоциация Оливковых Торговцев,64197274,41.04,buy,924-09-27 05:55,0,450.0,817.920000


In [5]:
# Calculate the average amnesty by guild
average_amnesty = trading.groupby('guild', as_index=False) \
    .agg({'amnesty': 'mean'}) \
    .rename(columns={'amnesty': 'avg_amnesty'})
average_amnesty.head(3)

,guild,avg_amnesty
0,Ассоциация Оливковых Торговцев,682.928338
1,Братство Виноделов,1726.452617
2,Братство Кузнецов,1835.734533


<b>Attention:</b> The ratio is calculated <b>not based on sum</b> but on the number of operations
$$Ratio = \frac{quantity_{sell}}{quantity_{buy}}$$
$$\overline{Amnesty_i}=\frac{1}{k}\sum_{j=1}^{k}Amnesty_{ij}$$
$$i-some~guild,~~~~k-number~of~guild~operations$$
$$Efficiency_i = Ratio_i * \overline{Amnesty_i}$$

In [6]:
guilds = trading \
    .pivot_table(index='guild', columns='countertype',
                 values='cost', aggfunc=pd.Series.count) \
    .reset_index()
guilds = guilds.merge(average_amnesty)

guilds['ratio'] = guilds.sell / guilds.buy
guilds['efficiency'] = guilds.ratio * guilds.avg_amnesty

# It is important to round to 2 decimal places
guilds = guilds \
    .sort_values('efficiency', ascending=False) \
    .round(2) \
    .head(1) \
    .reset_index(drop=True)
guilds

,guild,buy,sell,avg_amnesty,ratio,efficiency
0,Братство Кузнецов,6161,659,1835.73,0.11,196.36


In [7]:
# Filter data to include only entries with the guild 'Братство Кузнецов'
trading = trading[trading.guild == guilds.guild.iloc[0]]
# The date format '924-09-27 12:57' is hard to parse, so we check
# the data in the column and confirm that everything is in the same format.
# Therefore, we simply extract the hour substring from the string.
trading['hour'] = trading.date.str[10:12]
hour = trading.query('countertype == "sell"') \
    .hour \
    .value_counts() \
    .reset_index()[['hour']] \
    .head(1)

In [8]:
efficiency_guild = pd.concat([guilds, hour], axis=1)[['guild', 'efficiency', 'hour']]
efficiency_guild

,guild,efficiency,hour
0,Братство Кузнецов,196.36,21
